### key = 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D

### pt1 = 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D
### ct1 = 12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae
### ft1 = 1a c6 30 96 23 3a 40 96 6a 86 5c 4a 61 ed 99 2f

### pt2 = 0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D
### ct2 = f5 e9 bf e3 d6 1f 27 97 52 96 a3 ae 34 b1 ee 60

### pt3 = 0x1A, 0x1A, 0x1A, 0x1A, 0x2B, 0x2B, 0x2B, 0x2B, 0x3C, 0x3C, 0x3C, 0x3C, 0x4D, 0x4D, 0x4D, 0x4D
### ct3 = 7b fc cc ff ac 08 29 74 ac cd f5 22 c0 19 94 84

In [10]:
import chipwhisperer as cw
import time

version = cw.targets.SimpleSerial2
prog = cw.programmers.STM32FProgrammer


scope = cw.scope()
target = cw.target(scope, version)

scope.default_setup()





In [11]:
path = "../firmware/mcu/simpleserial-aes2/simpleserial-aes2-CWLITEARM.hex"

cw.program_target(scope, prog, path)

target.reset_comms()
target.flush()

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6495 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6495 bytes


In [12]:
text1 = bytearray([0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D])
text2 = bytearray([0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D, 0x4A, 0x3B, 0x2C, 0x1D])
text3 = bytearray([0x1A, 0x1A, 0x1A, 0x1A, 0x2B, 0x2B, 0x2B, 0x2B, 0x3C, 0x3C, 0x3C, 0x3C, 0x4D, 0x4D, 0x4D, 0x4D])

In [13]:
ct1 = bytearray.fromhex('12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae')
ct2 = bytearray.fromhex('f5 e9 bf e3 d6 1f 27 97 52 96 a3 ae 34 b1 ee 60')
ct3 = bytearray.fromhex('7b fc cc ff ac 08 29 74 ac cd f5 22 c0 19 94 84')

In [33]:
ct3

bytearray(b'{\xfc\xcc\xff\xac\x08)t\xac\xcd\xf5"\xc0\x19\x94\x84')

In [45]:
key = bytearray([ 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D, 0x1A, 0x2B, 0x3C, 0x4D])

In [46]:
#trace = cw.capture_trace(scope, target, text1)

In [47]:
#trace = cw.Trace(wave, textin, textout, key)

In [48]:
# import holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')
# curve = hv.Curve(trace.wave).opts(width=600, height=600)

In [49]:
target.flush()

In [32]:
target.simpleserial_write('b', text3)
valx = target.simpleserial_read_witherrors()
print(valx['payload'][0:16])

bytearray(b'$\xe8\x01b\xf59\x1fAaW\xca \x02loT')


In [16]:
valx['payload'][16:]

bytearray(b'{\xfc\xcc\xff\xac\x08)t\xac\xcd\xf5"\xc0\x19\x94\x84')

In [17]:
correct_input = bytearray.fromhex("a027b9210ac77ecaca16252fc02ef23f")

In [31]:
print(correct_input)

bytearray(b"\xa0\'\xb9!\n\xc7~\xca\xca\x16%/\xc0.\xf2?")


In [18]:
value1 = valx['payload'][0]
value2 = valx['payload'][1:16]
#eight_input = bytearray.fromhex("473ddb7ba3318a1010ffc24e1fc30425")
print(value1)
print(value2)

36
bytearray(b'\xe8\x01b\xf59\x1fAaW\xca \x02loT')


In [19]:
value1 == correct_input[0] and value2 == correct_input[1:16]

False

In [20]:
target.simpleserial_write('f', text3)
valx = target.simpleserial_read_witherrors()
print(valx)

{'valid': True, 'payload': CWbytearray(b'ff e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 a2 e7 10 0a a5 ad 97 60 44 de 1e 15 d0 a1 e1 57'), 'full_response': CWbytearray(b'00 72 20 ff e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 a2 e7 10 0a a5 ad 97 60 44 de 1e 15 d0 a1 e1 57 39 00'), 'rv': bytearray(b'\x00')}


In [21]:
valx['payload'][16:]

bytearray(b'\xa2\xe7\x10\n\xa5\xad\x97`D\xde\x1e\x15\xd0\xa1\xe1W')

In [22]:
bytearray(b'\x12\xe3\xfb6\x9a8z\xbc\x89\xaa\xc3\xaa\x14\x1d\x96\xae')
bytearray(b'\x02\xfe\xf8\x10\x01k\x0en\xcd\xee]\x18f\x9d-\x1b')
bytearray(b"\xf5\xe9\xbf\xe3\xd6\x1f\'\x97R\x96\xa3\xae4\xb1\xee`")
bytearray(b'\x8cK\xe9\xc5\xc1\x03\xe8\x06\xb3\xb2\x11\xdf\x1f\xd5\xc5\x01')

bytearray(b'\x8cK\xe9\xc5\xc1\x03\xe8\x06\xb3\xb2\x11\xdf\x1f\xd5\xc5\x01')

In [23]:
valx['payload'][0:16] == correct_input

False

We'll probably crash the target a few times while we're trying some glitching. Create a function to reset the target:

In [24]:
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()

### clk_src = "target"  or  "clkgen"
### offset = [-48.8, 48.8]
### width = [-50, 50]
### output = clock_xor
### ext_offset = 0
### scope.io.hs2 = "glitch"
### trigger_src = ext_single

In [25]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])

In [26]:
gc.glitch_plot(plotdots={"success":"+g", "reset":None, "normal":None}) #"xr"

:DynamicMap   []
   :Points   [width,offset]

setting ranges for each glitch setting:

storing the results

In [27]:
num_tries = 1

scope.io.hs2 = "glitch"
scope.glitch.clk_src = "clkgen"
scope.glitch.width = -10.15625 + 1
scope.glitch.offset = -39.84
scope.glitch.ext_offset = 5400
scope.glitch.repeat = 3
scope.glitch.trigger_src='ext_single'

gc.set_range("width", 30, 35) #value can be in (0, 48)
gc.set_range("offset", 8, 48)#values can be in (-48, 48)
gc.set_range("ext_offset", 5550, 5950)
gc.set_range("tries", 1, num_tries)
gc.set_global_step([1]) #gc.set_global_step([8, 4, 2, 1])

# gc.set_range("width", 3, 5)
# gc.set_range("offset", -13, -13)
# gc.set_range("ext_offset", 5550, 5950)
# gc.set_global_step(0.4)
# scope.glitch.repeat = 1

# scope.glitch.repeat = 1
# gc.set_step("tries", 1)
# scope.adc.timeout = 0.1
# gc.set_range("ext_offset", 0, 40)
# gc.set_step("ext_offset", 1)

### dir(gc)

In [28]:
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=30.0, continuous_update=False, description='width setting:', disabled=True, max=35.0, min=30…

FloatSlider(value=8.0, continuous_update=False, description='offset setting:', disabled=True, max=48.0, min=8.…

FloatSlider(value=5550.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=5950.…

FloatSlider(value=1.0, continuous_update=False, description='tries setting:', disabled=True, max=1.0, min=1.0,…

glitch settings

In [29]:
scope.glitch.clk_src = "clkgen" 
scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.io.hs2 = "glitch"

In [30]:
succesful_glitches = []

###  or simply
scope.cglitch_setup()

In [34]:
from tqdm.notebook import trange
import struct

#correct_input = bytearray.fromhex("a027b9210ac77ecaca16252fc02ef23f")
correct_input = bytearray(b'$\xe8\x01b\xf59\x1fAaW\xca \x02loT')
reboot_flush()
broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    scope.arm()

    #Do glitch loop

    target.simpleserial_write('b', text3)

    ret = scope.capture()
    
    loff = scope.glitch.offset
    lwid = scope.glitch.width
    lext = scope.glitch.ext_offset

    if ret:
        print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors(pay_len=32)
        if val['valid'] is False:
            gc.add("reset")
            print("invalid answer---discontinuing")
            continue
        else:
            if val['payload'] is None:
                print("valid answer but no payload---discontinuing")
                continue #what
            gcnt = val['payload'][0]
            ipval = val['payload'][1:16]
            value = val['payload'][16:]
            #print(value)
            #print(val['payload'])
            
            if (value != ct3) and (val['payload'][0:16] == correct_input): #for loop check
                broken = True
                gc.add("success")
                print(val['payload'])
                print("glitchded at",loff, lwid, lext)
                print("🐙")#, end="")
                succesful_glitches.append(val['payload'][16:])
                continue# <-- remove this to try for multiple glitches
            else:
                gc.add("normal")
                #print("glitched before 8th round input---discontinuing")
                continue



print("Done glitching")
#a0 27 b9 21 0a c7 7e ca ca 16 25 2f c0 2e f2 3f 12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae

invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 1a 07 03 36 15 27 81 0d e0 29 c2 f5 34 cd 25 10')
glitchded at 17.96875 30.078125 5562
🐙
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 14 4c 5b b5 57 a0 37 6c 5d 8e dd 31 1b 1e 1e 92')
glitchded at 17.96875 30.078125 5563
🐙
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7f 49 88 32 ca d9 2f 38 5f 0e e3 fc 44 ee 61 42')
glitchded at 17.96875 30.078125 5565
🐙
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b fc 8b ff ac 33 29

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


Timeout - no trigger
invalid answer---discontinuing
invalid answer---discontinuing


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


Timeout - no trigger
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---d

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


Timeout - no trigger
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b fc ac ff ac 99 29 74 49 cd f5 22 c0 19 94 e2')
glitchded at 16.015625 32.03125 5595
🐙
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b fc d8 ff ac 9e 29 74 01 cd f5 22 c0 19 94 ab')
glitchded at 16.015625 32.03125 5597
🐙
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b aa cc ff 25 08 29 74 ac cd f5 49 c0 19 c0 84')
glitchded at 16.015625 32.03125 5605
🐙
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b e5 cc ff 5f 08 29 74 ac cd f5 2c c0 19 20 84')
glitchded at 16.015625 32.03125 5611
🐙
CWbytearray(b'24 e8 01 62 f5 39 1f 41 61 57 ca 20 02 6c 6f 54 7b fb cc ff 25 08 29 74 ac cd f5 bf c0 19 ad 84')
glitchded at 16.015625 32.03125 5619
🐙
invalid answer---discontinuing
CWbytearray(b'24 e8 01 62 f5 39 1f 41 

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08


Timeout - no trigger
invalid answer---discontinuing


KeyboardInterrupt: 

In [1]:
glitched = bytearray.fromhex("1a c6 30 96 23 3a 40 96 6a 86 5c 4a 61 ed 99 2f")
print(glitched)
# ct3 = 7b fc cc ff ac 08 29 74 ac cd f5 22 c0 19 94 84

bytearray(b'\x1a\xc60\x96#:@\x96j\x86\\Ja\xed\x99/')


In [41]:
succesful_glitches

[bytearray(b"\x1a\x07\x036\x15\'\x81\r\xe0)\xc2\xf54\xcd%\x10"),
 bytearray(b'\x14L[\xb5W\xa07l]\x8e\xdd1\x1b\x1e\x1e\x92'),
 bytearray(b'\x7fI\x882\xca\xd9/8_\x0e\xe3\xfcD\xeeaB'),
 bytearray(b'{\xfc\x8b\xff\xac3)t\xbf\xcd\xf5"\xc0\x19\x94\xe5'),
 bytearray(b'{\xfcA\xff\xac\x9b)tc\xcd\xf5"\xc0\x19\x94\x95'),
 bytearray(b'\x06\n\x0c|7~\xf0\x10Yc\xa2\xb0\xb8[\x9c\xac'),
 bytearray(b'TT!\x83\xdd\x97G\x80\xc3t\xe5\x15\xf8\xbbn\x9c'),
 bytearray(b'{\x84\xcc\xff_\x08)t\xac\xcd\xf5\xdd\xc0\x19\xe9\x84'),
 bytearray(b'{\xe3\xcc\xff+\x08)t\xac\xcd\xf5\xf8\xc0\x19\xc5\x84'),
 bytearray(b'{\xfc\xccK\xac\x08\xc4t\xac\xaf\xf5"S\x19\x94\x84'),
 bytearray(b'\x84\nla7\xd6bp\xb4\xb4\x04\xb0\xb3\xd5\x9c\x1c'),
 bytearray(b'\x91q\x00\xd4\xe3\xed+T\xa9\xf6AE:RX&'),
 bytearray(b'{\xfc\x95r\xac\xe2\x90t\xcf\x1c\xf5"\xe2\x19\x94\x05'),
 bytearray(b'm\xfc\xcc\xd4\xac\x08+b\xac\xf6\xce":\x80\x94\x84'),
 bytearray(b'Q!<H\xc5\xb4Y\xfe\r\x11\xae\x87\xad1F_'),
 bytearray(b':\xbe\xcf\xa4\xd3\xb78\xd5lP\x90\x18\x86

In [26]:
# correct_input = bytearray.fromhex("dd b8 44 9f 1b fd 53 de 49 83 fa 4b 3d 15 df 13")

In [1]:
# succesful_glitches = [bytearray(b'\x1a\xc60\x96#:@\x96j\x86\\Ja\xed\x99/'),
#  bytearray(b'\x12\xe3\xfb{\x9a8\xff\xbc\x89\xf6\xc3\xaa5\x1d\x96\xae'),
#  bytearray(b'\x12\xe3\xbe6\x9a\xc8z\xbc\x08\xaa\xc3\xaa\x14\x1d\x96\xe0'),
#  bytearray(b'\x12\xe3w\xa5\x9a\t\xc5\xbcn\xa9\xc3\xaa^\x1d\x96\xde'),
#  bytearray(b'\x9e\xa3\x1fk\x01\xf2\xb8\xec\xac?\xee\t(\x18\x0c\xcc'),
#  bytearray(b'\xcb}\xce\xe2\xe2L\x81\x05\xebsk\xdaN\x80R\x07'),
#  bytearray(b'\x12\xfb\xfb6\xf58z\xbc\x89\xaa\xc35\x14\x1d\xdb\xae'),
#  bytearray(b'\x12\xa9\xfb6>8z\xbc\x89\xaa\xc3[\x14\x1d\xe3\xae'),
#  bytearray(b'\x9f\xe3\xfb!\x9a82\xf3\x89\xc7~\xaa;\xad\x96\xae'),
#  bytearray(b'\x12\xe3\xdb6\x9a\x0ez\xbc\xd5\xaa\xc3\xaa\x14\x1d\x96?'),
#  bytearray(b'\x88\xe4\x19\xf8\x8d; \x9e\x02\xa9^C\xe9\xa9g\x94'),
#  bytearray(b'\x12\xe3\xc56\x9a\xb9z\xbc8\xaa\xc3\xaa\x14\x1d\x96\xf0'),
#  bytearray(b'J\x07\xfb6"8z\x06\x89\xaa\xbfK\x14\xaa\xcf\xae'),
#  bytearray(b'\x00W\xda\xee\xbd|\x11\xdb\x9d\xcd\xd3O;\xf4\x92\xdf'),
#  bytearray(b'\xcd\xa4\x84\x00\xa9M \x9c\xf8A\xf6\x82\xd2\xeb,2'),
#  bytearray(b'\x12\xe3\xfb\x87\x9a8N\xbc\x89\x16\xc3\xaa\x81\x1d\x96\xae'),
#  bytearray(b'\xc6 R\xd7G\x15i\xb9g\xe5h\xa7\x98b\xbd\x1c'),
#  bytearray(b'F\xabC\xbb\x13\x078\x17\xd7\t\x9b3\xf7\xd6\xcf\xc8'),
#  bytearray(b'H\xdb\xbb\xec\xca\xc2A\xcd\xc0\x99!\xc0\x8c_\xdd}'),
#  bytearray(b'\x12\x0e\xfb6\'8z\xbc\x89\xaa\xc3\xa8\x14\x1d"\xae'),
#  bytearray(b'[\xe3\xfb6\x9a8zA\x89\xaa\x89\xaa\x14\xbd\x96\xae'),
#  bytearray(b'Z \xfc\xa4G\x8fTD\x8c\xc4\x8e\xa76\xda\xbdy'),
#  bytearray(b'Y9\xfd\x9aQ\xccF \xf3\xeb\xbc\xe3\xb0\xb3\xb8\xfa'),
#  bytearray(b'6\x00\xe7\x03\x0c\x0b\x84\x13\x1ed\x17\xfb2\x95Y\xcc'),
#  bytearray(b'\x05?\x059\xf0{\xcb\x18\xa1K8\xe8Q\xee\xec\xa8'),
#  bytearray(b'6c\xe7\x03e\x0b\x84\x13\x1ed\x17\x892\x95p\xcc'),
#  bytearray(b"H?\xf19\xf02\xcb\xcf\xbdKY\xe8Q\'\xec\n"),
#  bytearray(b'\xcdV\x17S\xa4/\xe8I\x18\xb8c\xefZ\xe4=\x00'),
#  bytearray(b'\xce\xe3\xfb6\x9a8zY\x89\xaaX\xaa\x14J\x96\xae'),
#  bytearray(b'\xb3\xe3Y\xea\x9a\xdc[E\xaf\xad\xd5\xaaA\x98\x96-'),
#  bytearray(b'\x12\xe3\xfb\xcf\x9a8\x08\xbc\x89\xa1\xc3\xaa\xac\x1d\x96\xae'),
#  bytearray(b'k\x8e\xde\x8dr\x10\xc2\xbb\xb2U\xa4\x04sxyE'),
#  bytearray(b'\x12D\xc8Vf\xd7\xf5\xbc^=\xc3t\xcd\x1d\t\x97'),
#  bytearray(b't$\x08\x98\x89Z\xaa\xf7$\xcf\xd7\x0fL\x88\x14['),
#  bytearray(b'f\xe3\xfb\xd8\x9a8W\xfc\x89\x0cF\xaa\xea\x83\x96\xae'),
#  bytearray(b'2\xde\x12\x95,\xd8\x80\xac\x82`G\xb8\xab\xa1g\x9c'),
#  bytearray(b'\x95\xc1\xca\xf2\x8e!\x84\x0e\xfbV\x84\xd2Z/\x14\x99'),
#  bytearray(b'\xb7\xe3\xfb6\x9a8zV\x89\xaa\xff\xaa\x14t\x96\xae'),
#  bytearray(b'\x12\xe3\xacX\x9a/V\xbc;\xa1\xc3\xaa\x00\x1d\x96\xd1'),
#  bytearray(b'\x12\xe3\xfb\x97\x9a8R\xbc\x89\xa3\xc3\xaa\x05\x1d\x96\xae'),
#  bytearray(b'M\xf9\x94\xd0\x96f\xbe\x98\xb2&\xd8\xbc\xd5\xf3BY'),
#  bytearray(b'3\xe3\xedo\x9a\x8fMd\xd4\xcd0\xaa\x90D\x96\x11'),
#  bytearray(b'O\x8cQ\xa7%++\xc5\xbb_.\r];Yn'),
#  bytearray(b"\xa2\'\x028\x92\x9aJ\xd0g\xf1\xa2\xa2\xa3\xc6\x90i"),
#  bytearray(b'\x0c)\xbd\xc3\xc0\x06\t\xc3v\x84\x15\x0b\xc6Q\xb9\xc8'),
#  bytearray(b"\xf8\'\xfb8\x928JO\x89\xf1?\xa2\xa3}\x90\xae"),
#  bytearray(b'\xf8\xe3\xfb6\x9a8zO\x89\xaa?\xaa\x14}\x96\xae'),
#  bytearray(b'3:\xc5!\x1ab24#\xc7\xc8\x88;\xdc\xadm'),
#  bytearray(b'\x12\xe3\xc06\x9awz\xbcZ\xaa\xc3\xaa\x14\x1d\x96H'),
#  bytearray(b'\x12\xe3J6\x9aSz\xbcS\xaa\xc3\xaa\x14\x1d\x96\xf5'),
#  bytearray(b'\x12\xe3J6\x9aSz\xbcS\xaa\xc3\xaa\x14\x1d\x96\xf5'),
#  bytearray(b'\x12\xe3\xa26\x9a\xd8z\xbc\x0f\xaa\xc3\xaa\x14\x1d\x96F'),
#  bytearray(b'\x12%\xfb6\x1e8z\xbc\x89\xaa\xc3\x83\x14\x1dd\xae'),
#  bytearray(b'\xf3\xe3\xfb6\x9a8z\xbf\x89\xaaf\xaa\x14\xcc\x96\xae'),
#  bytearray(b'\xea\xe3\xfb6\x9a8zr\x89\xaa\xfe\xaa\x14y\x96\xae'),
#  bytearray(b'\x07\xe3\xfb6\x9a8z\xa1\x89\xaa\xcd\xaa\x14\xec\x96\xae'),
#  bytearray(b'\x12\xe3\xfb\xdf\x9a8\xdd\xbc\x89\xd6\xc3\xaa\xcd\x1d\x96\xae'),
#  bytearray(b'\xd8\xdf\xb2\xf7\xf2J\xc2U\x1f\xfe\x17\xc3\xe9\xab\xb9\x05'),
#  bytearray(b"\xc6 R\xa2G\x15\x1a\xb9g\'h\xa7\xdeb\xbd\x1c"),
#  bytearray(b' \xf5{\xa4u\xeeT\xa8\xea\xc4\xb1\xf76D]H'),
#  bytearray(b"\xf1\xae\x84\xa2\x1dM\x1a\x8a\xf8\'\xbc\x95\xde\x1e+2"),
#  bytearray(b'\x12\xe3\xfb\xbc\x9a8q\xbc\x89\x87\xc3\xaa\xf0\x1d\x96\xae'),
#  bytearray(b'\x12\xa4\x886\xa9}z\xbc\x1b\xaa\xc3\x82\x14\x1d,M'),
#  bytearray(b'};\x94\x07W$\x87h\xc8\xd1\xa3\xc9\xd4"\x10\x84'),
#  bytearray(b'\x12V\xfb6R8z\xbc\x89\xaa\xc3\xc1\x14\x1d\x84\xae'),
#  bytearray(b'\x12z\xfb628z\xbc\x89\xaa\xc3\xd0\x14\x1dd\xae'),
#  bytearray(b'\x12\xe3\x8e6\x9a\x8dz\xbc\x8f\xaa\xc3\xaa\x14\x1d\x96\x91'),
#  bytearray(b'\xc8\xe3\xfb6\x9a8z\xa4\x89\xaa\xfa\xaa\x14C\x96\xae'),
#  bytearray(b'k\x8e\xde\x8dr\x10\xc2\xbb\xb2U\xa4\x04sxyE'),
#  bytearray(b'\x12\xb5P^\x18|\xbf\xbc/\xf0\xc3\xc8\xc1\x1dQ\xf5'),
#  bytearray(b'\x12\x92:\xef\xferP\xbc\ns\xc3\x00T\x1d\xa5\xb6'),
#  bytearray(b'\x12\xeb\xfb6\xd28z\xbc\x89\xaa\xc3N\x14\x1dF\xae'),
#  bytearray(b'6\x00\xe7\x03\x0c\x0b\x84\x13\x1ed\x17\xfb2\x95Y\xcc'),
#  bytearray(b'\xa6\xf3\x8fT\xf0:\xc9X)\xd4\x01>0h\x80f'),
#  bytearray(b'\x95\xe3p\x1e\x9a\xf0\xa7\xd5\xcd\xf2>\xaaz\n\x96\x1f'),
#  bytearray(b'\x12\xe3,6\x9a\xa4z\xbc\xfc\xaa\xc3\xaa\x14\x1d\x96~'),
#  bytearray(b'FK\xbe!\xb8\xb1\xfc\x87;f\xd4\xe6\xd2}\x801'),
#  bytearray(b'\xcc\x90N\xa6s2s=\xf4\xccm\t*\x00-\xde'),
#  bytearray(b'\xa1\xe3\xfb\xe7\x9a8w?\x89\xc1\x19\xaa:B\x96\xae'),
#  bytearray(b'\x12\xe3\xfbs\x9a8a\xbc\x89O\xc3\xaa\xc9\x1d\x96\xae'),
#  bytearray(b'\x99\x1e$\xe9\xde\x9c\xee\xb4\xabw\x1eDY\x0f\xbd\x97'),
#  bytearray(b'\x12\xe3\x88\x7f\x9a}\xa1\xbc\x1b\x80\xc3\xaa\x11\x1d\x96M'),
#  bytearray(b'\x19\xc1R\xd3\x8e>\x0c\x00\xe1S\x17\xd2\xdb\xea\x14\x8a'),
#  bytearray(b"\x13\xaa\'\xef\xf1\x9bP-\xf6s\x1b\xcbT\xfe\x86S"),
#  bytearray(b'\x12\xe3\xfbo\x9a8M\xbc\x89\xcd\xc3\xaa\x90\x1d\x96\xae'),
#  bytearray(b'3\xe3wo\x9a\tMdn\xcd0\xaa\x90D\x96\xde'),
#  bytearray(b'M\x1bw6\xbe\tz\x98n\xaa\xd8!\x14\xf3"\xde'),
#  bytearray(b'\x96.T5HX\xe4\x82\xaa\xf7\x10\xf5\r_\xda/'),
#  bytearray(b'\x12\xe3\xed6\x9a\x8fz\xbc\xd4\xaa\xc3\xaa\x14\x1d\x96\x11'),
#  bytearray(b'\x123\xbe6@\xc8z\xbc\x08\xaa\xc3\x90\x14\x1d\xc7\xe0'),
#  bytearray(b'\x16I{k\x1f\xb2\xb8\xe8t?\xbbm(\x072\x12'),
#  bytearray(b'\xa6\x8eJ\xdb|S\xe6XSd\x01-ih \xf5'),
#  bytearray(b'\xcb\x9b\xfa5\xb6\x92\xe4\x05\xca\xf7k+\r\x80\xc4\xe4'),
#  bytearray(b'\x12\x04\xfb6\xfe8z\xbc\x89\xaa\xc3,\x14\x1dJ\xae'),
#  bytearray(b'\x12\xe3\xe96\x9a(z\xbc<\xaa\xc3\xaa\x14\x1d\x96e'),
#  bytearray(b'\xe9Q\x17\x1bF\xd3\x1fK2~h*%\x16\x88\xad'),
#  bytearray(b'\xfc\x00^.5G\xb4\xf4\x9eYl\xbcO\x1e\x00\xc2'),
#  bytearray(b'\x12\xe3\xd46\x9a\xc2z\xbc\xd6\xaa\xc3\xaa\x14\x1d\x96\xed'),
#  bytearray(b'J\xbe\xd26\xe1\x0ez\x06V\xaa\xbf6\x14\xaa,\xce'),
#  bytearray(b'\x12\x1a\xfb6\xcb8z\xbc\x89\xaa\xc3\x15\x14\x1dS\xae'),
#  bytearray(b'\xbf-\xfe;\xab\xa2\x1c\xedf\xb7\xc8\xd6\xed\xa3\xd6t'),
#  bytearray(b'\xbd\xb8\xed"\xc0f\x02m_\xe3f/\x057E\xc6'),
#  bytearray(b'\xf1W\xc5\x0eK&\xb5\x8a\x01\xa3\xbc#\xe7\x1e\xa0\x95'),
#  bytearray(b'\x12\xe3\xfb8\x9a8\xca\xbc\x89K\xc3\xaa\xaf\x1d\x96\xae'),
#  bytearray(b'\x12\xa4\x8e6\xa9\x8dz\xbc\x8f\xaa\xc3\x82\x14\x1d,\x91'),
#  bytearray(b'\x12\xc7\xfb6\x9f8z\xbc\x89\xaa\xc3 \x14\x1d%\xae'),
#  bytearray(b'\x12\xa4\xfb6\xa98z\xbc\x89\xaa\xc3\x82\x14\x1d,\xae'),
#  bytearray(b'k\x8e\xde\x8dr\x10\xc2\xbb\xb2U\xa4\x04sxyE'),
#  bytearray(b'_F\xf5\x8ek\x8cb\x91\x93h\xa2\xd9<\xef\x19z'),
#  bytearray(b'\x12\x92:r\xfer\xd8\xbc\n\x8a\xc3\x00\x1a\x1d\xa5\xb6'),
#  bytearray(b'y\x9b\xf7\xfe\xb6\xe6\x19g\xc9\xadp+g\x9a\xc4\xdc'),
#  bytearray(b'\xccC\x8d\x99\x08\x9f\x9c=a\xa3m\x9c\xc0\x00\xfa\xdd'),
#  bytearray(b'\xccC\xe7\xa6\x08\x0bs=\x1e\xccm\x9c*\x00\xfa\xcc'),
#  bytearray(b'\x12\xe3n6\x9a;z\xbc\xb2\xaa\xc3\xaa\x14\x1d\x96\x84'),
#  bytearray(b'\xcb7r\xbd^\x1d\xe5\xc7\x95V\x87Kc\xe3\x1f\xdb'),
#  bytearray(b'&k\xe5\xe7\xf1\xc0\x87\xa2\xcf\xabP\xd0K\x0f\x85('),
#  bytearray(b'\xa8_k{\xa0\xd4\x81(\xf6\x1d\x10gXp\x1em'),
#  bytearray(b'\x12\xe3,6\x9a\xa4z\xbc\xfc\xaa\xc3\xaa\x14\x1d\x96~'),
#  bytearray(b'\xbfL\xfb\xe7\xaf8w,\x89\xc1Q\xe3:\xd7\xab\xae'),
#  bytearray(b'D\x1e``\x15\x1f\xa7}\xe0\xbf\xc8]\x9a\xe1\xf2['),
#  bytearray(b'\x12\xe3\xfbo\x9a8M\xbc\x89\xcd\xc3\xaa\x90\x1d\x96\xae'),
#  bytearray(b"\xcc\x9e\'\xda\xeb\xfe\x12\xd8\xe5\xcdR\xb2\x1c\xbd\xd5x"),
#  bytearray(b'\x12\xe3\xfbK\x9a8\xc3\xbc\x89\x98\xc3\xaa\xee\x1d\x96\xae'),
#  bytearray(b"\x12\xe3\xfb\'\x9a8<\xbc\x89H\xc3\xaa\xc1\x1d\x96\xae"),
#  bytearray(b'\xff\xf3\xfb6\xf08z\xe5\x89\xaa\xa6>\x14U\x80\xae'),
#  bytearray(b'\x12\xb9\xfb6\xf68z\xbc\x89\xaa\xc3\xf9\x14\x1d\xed\xae'),
#  bytearray(b'\xbf\xe3\xfb\xe7\x9a8w,\x89\xc1Q\xaa:\xd7\x96\xae'),
#  bytearray(b'\x12\x8b\xfb6u8z\xbc\x89\xaa\xc3r\x14\x1d\xf1\xae'),
#  bytearray(b'\x18\xe3\xfb6\x9a8z\xe8\x89\xaa\xea\xaa\x14I\x96\xae'),
#  bytearray(b'\xccC\x8d\x99\x08\x9f\x9c=a\xa3m\x9c\xc0\x00\xfa\xdd'),
#  bytearray(b'\x124\xfb6W8z\xbc\x89\xaa\xc3\xea\x14\x1d\xfd\xae'),
#  bytearray(b'\x12\xe3\x8a6\x9a\x1ez\xbc\xac\xaa\xc3\xaa\x14\x1d\x96,'),
#  bytearray(b'\x13elg\xfd\n\x80\xb8\x8e\x8c\xc8]\xa3#j\\'),
#  bytearray(b'X\x89\x90o\xdb\xceM]\xaa\xcd\xfa\xc5\x90\xd9\x87G'),
#  bytearray(b"\'W\xfb6\x858z\x07\x89\xaa<X\x14\x1a\x12\xae"),
#  bytearray(b'\x12\xe3\xfb#\x9a8\xc1\xbc\x89P\xc3\xaar\x1d\x96\xae'),
#  bytearray(b'3\xe3\xfbo\x9a8Md\x89\xcd0\xaa\x90D\x96\xae'),
#  bytearray(b"\x12\'\xfb6\x928z\xbc\x89\xaa\xc3\xa2\x14\x1d\x90\xae"),
#  bytearray(b'o\xda\x03tZ\xd1{\n\xa9\xae\xb2S\xa5c\xa19'),
#  bytearray(b'\xfa\xbb4\x98\x06j\xaa\xab\x0f\xcf\x94\xc8L\xef\x00\x8b'),
#  bytearray(b'\xc3\xbd\x88\xbasT:\xed!\x0fQ\x9c\xdd\x93\xedO'),
#  bytearray(b'\xb3\xcc\x8d_\xb8D\x99\x95\x02\xbe\xb7\xe7"\xa1\xa6\xfd'),
#  bytearray(b'\xee\\\xff\xf8C\xef\xe0|\x14\xfaE&\x80\x11^\x8e'),
#  bytearray(b'\xf8\xe1\xfa5\xd5\x92\xe4O\xca\xf7?[\r}\xb6\xe4'),
#  bytearray(b'Z\xe3\xfb6\x9a8z\x86\x89\xaas\xaa\x14)\x96\xae'),
#  bytearray(b'\x9f\xe3\xe9!\x9a(2\xf3<\xc7~\xaa;\xad\x96e'),
#  bytearray(b'\x12\xe3\xce6\x9aLz\xbc\xeb\xaa\xc3\xaa\x14\x1d\x96\x07'),
#  bytearray(b'\xe9u\x17\x1b\x81\xd3\x1fK2~h\xf4%\x16[\xad'),
#  bytearray(b"\x91\x7f\xfb6r8z\xf7\x89\xaarB\x14\'\x1b\xae"),
#  bytearray(b'J\xe3\xfb6\x9a8z\x06\x89\xaa\xbf\xaa\x14\xaa\x96\xae'),
#  bytearray(b'\x12\xe3\xfb\xb0\x9a8\x16\xbc\x89|\xc3\xaa\xb9\x1d\x96\xae'),
#  bytearray(b'\xcd\xa4\x84\x00\xa9M \x9c\xf8A\xf6\x82\xd2\xeb,2'),
#  bytearray(b'\xae\xe3\xfb\xfb\x9a8\x07\xea\x89\xac\xd0\xaa\x12\xd3\x96\xae'),
#  bytearray(b'\xca?\x84\xe7\x1cM\xe7F\xf8v\xc1\xa5\x841\xfa2'),
#  bytearray(b'\x12m\xfb6\x008z\xbc\x89\xaa\xc3\xb7\x14\x1d\xc9\xae'),
#  bytearray(b'\x12\xe3\x846\x9aMz\xbc\xf8\xaa\xc3\xaa\x14\x1d\x962'),
#  bytearray(b'\x12\xe3\x126\x9a\x93z\xbc\x0f\xaa\xc3\xaa\x14\x1d\x96g'),
#  bytearray(b'\x12\xa4\x8e6\xa9\x8dz\xbc\x8f\xaa\xc3\x82\x14\x1d,\x91'),
#  bytearray(b'-\xe3\xfb6\x9a8z-\x89\xaaj\xaa\x14^\x96\xae'),
#  bytearray(b"\x02\xe3\xfb6\x9a8z\'\x89\xaa\xb2\xaa\x14\xfb\x96\xae"),
#  bytearray(b'\x08\xa0\xf9\x9cn\x14\xc9\xd6\x1f\x80j\xf8\xd8\xe8\x8e\x01'),
#  bytearray(b'8u\x07\x1a\xd6{\x87y\x16<^\x9c\xf3\xc3j\xfb'),
#  bytearray(b'\x12\xa0\xf9rn\x14\xd8\xbc\x1f\x8a\xc3\xf8\x1a\x1d\x8e\x01'),
#  bytearray(b'\x12\x92:6\xferz\xbc\n\xaa\xc3\x00\x14\x1d\xa5\xb6'),
#  bytearray(b']\xe3p\x1e\x9a\xf0\xa7\xb2\xcd\xf2\x12\xaaz\xe1\x96\x1f'),
#  bytearray(b'\xccC\xe7\x03\x08\x0b\x84=\x1edm\x9c2\x00\xfa\xcc'),
#  bytearray(b'&kV?\xf1\xd1\xa5\xa2\x11DP\xd0\x15\x0f\x85h'),
#  bytearray(b'FKw!\xb82\xfc\x87\xc3f\xd4\xe6\xd2}\x80N'),
#  bytearray(b'\x12\xd9\xfb698z\xbc\x89\xaa\xc3\xfa\x14\x1d\xd1\xae'),
#  bytearray(b'\x00\x8a\xfb\x0f+8\xd7\xdb\x89\xbd\xd3l\x84\xf4\n\xae'),
#  bytearray(b"\x91\x95I\xee,Z\x11\xf7k\xcdrY;\'=\xba"),
#  bytearray(b'\x12\xe3\xfb\xfa\x9a8\x97\xbc\x89\xb8\xc3\xaaL\x1d\x96\xae'),
#  bytearray(b'\x9a\xe3\xfb\xae\x9a8|h\x89\xdfm\xaa6Y\x96\xae'),
#  bytearray(b'3Pp\x1e\xf8\xf0\xa74\xcd\xf2\xc8@z\xdc\n\x1f'),
#  bytearray(b'3\xe3p\x1e\x9a\xf0\xa74\xcd\xf2\xc8\xaaz\xdc\x96\x1f'),
#  bytearray(b'\x814\x7f}Wd\x87\x82\x07e\t\xeab\\\xfd\x81'),
#  bytearray(b'\x12\xe3\xfbo\x9a8M\xbc\x89\xcd\xc3\xaa\x90\x1d\x96\xae'),
#  bytearray(b'\x12\xe3\xbes\x9a\xc8a\xbc\x08O\xc3\xaa\xc9\x1d\x96\xe0'),
#  bytearray(b'L\xa5\x97o\x13\xd3M\x163\xcd*K\x90Rq.'),
#  bytearray(b'\x12\xe3w6\x9a\tz\xbcn\xaa\xc3\xaa\x14\x1d\x96\xde'),
#  bytearray(b'\x123\xfb6@8z\xbc\x89\xaa\xc3\x90\x14\x1d\xc7\xae'),
#  bytearray(b'\xcb9\xe1\\\xac\xc3\x8e\x05C\xd8kR\x8b\x807\n'),
#  bytearray(b'\x12\xe3\xfb$\x9a8\x10\xbc\x89]\xc3\xaa\xeb\x1d\x96\xae'),
#  bytearray(b'\r\x1c\x93\x80g>\xe6\xf2\xf9\x1ae\xa3\xa8\xfd\x96\xc1'),
#  bytearray(b'\x12\xe3E6\x9a\xc7z\xbcv\xaa\xc3\xaa\x14\x1d\x96@'),
#  bytearray(b'\x12\xe3\xc56\x9a\xb9z\xbc8\xaa\xc3\xaa\x14\x1d\x96\xf0'),
#  bytearray(b'\x12\xbe\xc76\xe1(z\xbc\xcf\xaa\xc36\x14\x1d,R'),
#  bytearray(b'\x12\xf1\xfb6y8z\xbc\x89\xaa\xc3\xc4\x14\x1d\xd6\xae'),
#  bytearray(b'\x12\xbeE6\xe1\xc7z\xbcv\xaa\xc36\x14\x1d,@'),
#  bytearray(b'\x12\xe3E6\x9a\xc7z\xbcv\xaa\xc3\xaa\x14\x1d\x96@'),
#  bytearray(b'\x00\xe3\xfb6\x9a8z\xdb\x89\xaa\xd3\xaa\x14\xf4\x96\xae'),
#  bytearray(b'\x94\xf7W\x13\x18h\xfc"3B\xf1\xff\xa9g\xc0\xbb'),
#  bytearray(b'\xa4K\xbf\x08\xad\x80\x03\x95\x1a\x14\xd5\x19\xd5\x08\x90\x18'),
#  bytearray(b'\xf1\xc6\x03e\x94\xdds\x8awr\xbc\xd8\xdc\x1el\xb1'),
#  bytearray(b'\x12\xe3\x846\x9aMz\xbc\xf8\xaa\xc3\xaa\x14\x1d\x962'),
#  bytearray(b'\x12\xaf\xfb6\x858z\xbc\x89\xaa\xc3T\x14\x1d\x8c\xae'),
#  bytearray(b'\xfb\x92:\x16\xferx\x05\n\x96\xba\x00j\x04\xa5\xb6'),
#  bytearray(b'\x12\x92:r\xfer\xd8\xbc\n\x8a\xc3\x00\x1a\x1d\xa5\xb6'),
#  bytearray(b'\x12\xe3\t6\x9a)z\xbc\xbc\xaa\xc3\xaa\x14\x1d\x96\xd5'),
#  bytearray(b'\x12\x92\xa76\xfe\x1bz\xbc\x8b\xaa\xc3\x00\x14\x1d\xa5\x9d'),
#  bytearray(b'\x12\xd2\xfb6T8z\xbc\x89\xaa\xc36\x14\x1d6\xae'),
#  bytearray(b'\x00\xe3\xbd\x16\x9a\x06\xad\xddv\x8e\xcc\xaac\xbc\x96\xc8'),
#  bytearray(b'\x05?\x059\xf0{\xcb\x18\xa1K8\xe8Q\xee\xec\xa8'),
#  bytearray(b'\xccC\xe7\x03\x08\x0b\x84=\x1edm\x9c2\x00\xfa\xcc'),
#  bytearray(b'\x12\xe3\xfb\xc9\x9a8\xa2\xbc\x89\xe8\xc3\xaa\xd8\x1d\x96\xae'),
#  bytearray(b'\x12\xe3\x9f6\x9a\x14z\xbc\xda\xaa\xc3\xaa\x14\x1d\x96\x99'),
#  bytearray(b'\xccC\xf6\xa6\x08\xf1s=\x0b\xccm\x9c*\x00\xfa\xf6'),
#  bytearray(b'\x12\xe3\xfb\xc5\x9a8H\xbc\x89M\xc3\xaa\n\x1d\x96\xae'),
#  bytearray(b'\x0e$\xfb\x10\xf28\xb7\xc0\x89\x7f\xfb\x9es\xb8\xf5\xae'),
#  bytearray(b'h\xe3\xfb6\x9a8z\xdd\x89\xaa3\xaa\x14\xfd\x96\xae'),
#  bytearray(b'@L\x12\xd1L\x93\x13C\x0fY\xa3\xf09\xb0\xa7g'),
#  bytearray(b'\x12\xbe\xe86\xe1Vz\xbcm\xaa\xc36\x14\x1d,#'),
#  bytearray(b']\xe3p\x1e\x9a\xf0\xa7\xb2\xcd\xf2\x12\xaaz\xe1\x96\x1f'),
#  bytearray(b'|\xfcp\x1e\xaa\xf0\xa7\x18\xcd\xf2L\x04z\x9e\xcb\x1f'),
#  bytearray(b'\x12\xe3\x146\x9anz\xbcG\xaa\xc3\xaa\x14\x1d\x96\xb4')]

In [42]:
import numpy as np
from collections import OrderedDict
import itertools

class DFA:

    finalkey = {}
    # Constructor
    def __init__(self, ciphertext_1, faultyone, ciphertext_2, faultytwo):
        self.ciphertext_1 = ciphertext_1
        self.faultyone = faultyone
        self.ciphertext_2 = ciphertext_2
        self.faultytwo = faultytwo 

    def __repr__(self):
        return f"DFA(ciphertext_1='{self.ciphertext_1}\n', faultyone='{self.faultyone}\n', ciphertext_2='{self.ciphertext_2}\n', faultytwo='{self.faultytwo}\n')"

    
    def dfa(self,i1, i2, i3, i4, f1, f2, f3, f4):

        Si = [  0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
                0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
                0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
                0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
                0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
                0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
                0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
                0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
                0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
                0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
                0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
                0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
                0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
                0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
                0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
                0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]


        d = {}
        key = ''
    
        d2 = {}
        key2 = ''

        guess1 = []
        guess2 = []

        
        def fxtime(tf,i):
        
            def xtime(integer):
                buffer=0
                if(integer & 0x80):#MSB ==1
                    buffer=(((integer << 1)^0x1b)&0xFF)
                else:
                    buffer= integer << 1
                return(buffer)
            
            if(tf == 1):
                return i
            if(tf == 2):
                return xtime(i)
            if(tf == 3):
                return xtime(i)^i
        
        for i in range(256):
            for j in range(256):
                temp = []
                if(fxtime(f1, i) == (Si[int(ciphertext_1[i1],16) ^ j] ^ Si[int(faultyone[i1],16) ^ j])):
                    key = str(i)+' '+str(j)
                    temp.append([j])
                    
                    tempk7 = []
                    for k in range(256):  #k
                        if(fxtime(f2, i) == (Si[int(ciphertext_1[i2],16) ^ k] ^ Si[int(faultyone[i2],16) ^ k])):
                            tempk7.append(k)
                    temp.append(tempk7)
                    
                    tempk10 = []
                    for l in range(256):
                        if(fxtime(f3, i) == (Si[int(ciphertext_1[i3],16) ^ l] ^ Si[int(faultyone[i3],16) ^ l])):
                            tempk10.append(l)
                    temp.append(tempk10)
                    
                    tempk13 = []
                    for m in range(256):
                        if(fxtime(f4, i) == (Si[int(ciphertext_1[i4],16) ^ m] ^ Si[int(faultyone[i4],16) ^ m])):
                            tempk13.append(m)
                    temp.append(tempk13)
                    d[key] = temp
        
        for i in range(256):
            for j in range(256):
                temp = []
                if(fxtime(f1, i) == (Si[int(ciphertext_2[i1],16) ^ j] ^ Si[int(faultytwo[i1],16) ^ j])):
                    key2 = str(i)+' '+str(j)
                    temp.append([j])
                    
                    tempk7 = []
                    for k in range(256): 
                        if(fxtime(f2, i) == (Si[int(ciphertext_2[i2],16) ^ k] ^ Si[int(faultytwo[i2],16) ^ k])):
                            tempk7.append(k)
                    temp.append(tempk7)
                    
                    tempk10 = []
                    for l in range(256):
                        if(fxtime(f3, i) == (Si[int(ciphertext_2[i3],16) ^ l] ^ Si[int(faultytwo[i3],16) ^ l])):
                            tempk10.append(l)
                    temp.append(tempk10)
                    
                    tempk13 = []
                    for m in range(256):
                        if(fxtime(f4, i) == (Si[int(ciphertext_2[i4],16) ^ m] ^ Si[int(faultytwo[i4],16) ^ m])):
                            tempk13.append(m)
                    temp.append(tempk13)
                    d2[key2] = temp
        
        for i in d.values():
            flag = 0
            for j in i:
                if(len(j) == 0):
                    flag = 1
            if(flag == 0):
                guess1.extend(list(itertools.product(*i)))


        for i in d2.values():
            flag = 0
            for j in i:
                if(len(j) == 0):
                    flag = 1
            if(flag == 0):
                guess2.extend(list(itertools.product(*i)))


        tkey = (list(set(guess1) & set(guess2)))
        
        if(tkey):
            tk = list(tkey[0])
            # print(tk)
            self.finalkey[i1] = hex(tk[0]) 
            self.finalkey[i2] = hex(tk[1])
            self.finalkey[i3] = hex(tk[2])
            self.finalkey[i4] = hex(tk[3])
            # for i in tk:
            #     print(hex(i))
    
    def getkey(self):
        tindices = [[0, 13,10,7], [4, 1, 14, 11], [8,5,2,15], [12,9,6,3]]
        findices = [[2, 1, 1, 3], [1, 1, 3, 2], [1, 3, 2, 1], [3, 2, 1, 1]]

        self.dfa(tindices[0][0], tindices[0][1], tindices[0][2], tindices[0][3], findices[0][0], findices[0][1], findices[0][2], findices[0][3])
        self.dfa(tindices[1][0], tindices[1][1], tindices[1][2], tindices[1][3], findices[1][0], findices[1][1], findices[1][2], findices[1][3])
        self.dfa(tindices[2][0], tindices[2][1], tindices[2][2], tindices[2][3], findices[2][0], findices[2][1], findices[2][2], findices[2][3])
        self.dfa(tindices[3][0], tindices[3][1], tindices[3][2], tindices[3][3], findices[3][0], findices[3][1], findices[3][2], findices[3][3])

        output = ",".join(value[2:] for value in OrderedDict(sorted(self.finalkey.items())).values())
        if(len(output) == 0):
            print("No key found")
        return output



def format_bytearray_to_hex(byte_array):
    return [f"0x{byte:02x}" for byte in byte_array]




# ciphertext_1 = format_bytearray_to_hex(eval(input("Enter the ciphertext_1 bytearray in the format 'bytearray(b'...')': ")))
# faultyone = format_bytearray_to_hex(eval(input("Enter the faulty1 bytearray in the format 'bytearray(b'...')': ")))

# ciphertext_2 = format_bytearray_to_hex(eval(input("Enter the ciphertext_2 bytearray in the format 'bytearray(b'...')': ")))
# faultytwo = format_bytearray_to_hex(eval(input("Enter the faulty2 bytearray in the format 'bytearray(b'...')': ")))

# ciphertext_1 = array[0]
# faultyone = array[1]
# ciphertext_2 = array[2]
# faultytwo = array[3]

# print(ciphertext_1)
# print(faultyone)
# print(ciphertext_2)
# print(faultytwo)

# new_dfa = DFA(ciphertext_1, faultyone, ciphertext_2, faultytwo)
# print(new_dfa.getkey())






In [43]:
text1 = bytearray(b'\x12\xe3\xfb6\x9a8z\xbc\x89\xaa\xc3\xaa\x14\x1d\x96\xae')
#text2 = bytearray(b'\x02\xfe\xf8\x10\x01k\x0en\xcd\xee]\x18f\x9d-\x1b')
text2 = bytearray(b'\x1a\xc60\x96#:@\x96j\x86\\Ja\xed\x99/')

text3 = bytearray(b'{\xfc\xcc\xff\xac\x08)t\xac\xcd\xf5"\xc0\x19\x94\x84')
text4 = bytearray(b'\xa2\xe7\x10\n\xa5\xad\x97`D\xde\x1e\x15\xd0\xa1\xe1W')

text5 = bytearray(b"\xf5\xe9\xbf\xe3\xd6\x1f\'\x97R\x96\xa3\xae4\xb1\xee`")
text6 = bytearray(b'\x8cK\xe9\xc5\xc1\x03\xe8\x06\xb3\xb2\x11\xdf\x1f\xd5\xc5\x01')

In [44]:
succesful_glitches1 = [ bytearray(b'\xa2\xe7\x10\n\xa5\xad\x97`D\xde\x1e\x15\xd0\xa1\xe1W')]

In [45]:
array = [0,0,0,0]
array[0] = format_bytearray_to_hex(text1)
array[1] = format_bytearray_to_hex(text2)
array[2] = format_bytearray_to_hex(text3)
array[3] = format_bytearray_to_hex(text4)
i = 0

ciphertext_1 = array[0]
faultyone = array[1]
ciphertext_2 = array[2]
#faultytwo = array[3]

# ned = DFA(ciphertext_1, faultyone, ciphertext_2, faultytwo)
# print(ned.getkey())

In [ ]:
value = succesful_glitches[4]
#array[1] = format_bytearray_to_hex(value)
#ciphertext_1 = array[0]
faultytwo = format_bytearray_to_hex(value)#array[1]
# ciphertext_2 = array[2]
# faultytwo = array[3]
# print(array[0])
# print(array[1])
# print(array[2])
# print(array[3])
#i= i+1
#print(i)
new_dfa = DFA(ciphertext_1, faultyone, ciphertext_2, faultytwo)
key = new_dfa.getkey()
print(key)
if len(key)!=0:
    print(value)
    

In [31]:
print(array)

[['0x12', '0xe3', '0xfb', '0x36', '0x9a', '0x38', '0x7a', '0xbc', '0x89', '0xaa', '0xc3', '0xaa', '0x14', '0x1d', '0x96', '0xae'], ['0x02', '0xfe', '0xf8', '0x10', '0x01', '0x6b', '0x0e', '0x6e', '0xcd', '0xee', '0x5d', '0x18', '0x66', '0x9d', '0x2d', '0x1b'], ['0x7b', '0xfc', '0xcc', '0xff', '0xac', '0x08', '0x29', '0x74', '0xac', '0xcd', '0xf5', '0x22', '0xc0', '0x19', '0x94', '0x84'], ['0xa2', '0xe7', '0x10', '0x0a', '0xa5', '0xad', '0x97', '0x60', '0x44', '0xde', '0x1e', '0x15', '0xd0', '0xa1', '0xe1', '0x57']]


In [110]:
array2 = [['0x12', '0xe3', '0xfb', '0x36', '0x9a', '0x38', '0x7a', '0xbc', '0x89', '0xaa', '0xc3', '0xaa', '0x14', '0x1d', '0x96', '0xae'], ['0x02', '0xfe', '0xf8', '0x10', '0x01', '0x6b', '0x0e', '0x6e', '0xcd', '0xee', '0x5d', '0x18', '0x66', '0x9d', '0x2d', '0x1b'], ['0x7b', '0xfc', '0xcc', '0xff', '0xac', '0x08', '0x29', '0x74', '0xac', '0xcd', '0xf5', '0x22', '0xc0', '0x19', '0x94', '0x84'], ['0xa2', '0xe7', '0x10', '0x0a', '0xa5', '0xad', '0x97', '0x60', '0x44', '0xde', '0x1e', '0x15', '0xd0', '0xa1', '0xe1', '0x57']]

In [111]:
array == array2

True

In [108]:
array = [0,0,0,0]
array[0] = format_bytearray_to_hex(text1)
array[1] = format_bytearray_to_hex(correct_input)
array[2] = format_bytearray_to_hex(text3)
array[3] = format_bytearray_to_hex(text4)
print(array)

[['0x12', '0xe3', '0xfb', '0x36', '0x9a', '0x38', '0x7a', '0xbc', '0x89', '0xaa', '0xc3', '0xaa', '0x14', '0x1d', '0x96', '0xae'], ['0xa0', '0x27', '0xb9', '0x21', '0x0a', '0xc7', '0x7e', '0xca', '0xca', '0x16', '0x25', '0x2f', '0xc0', '0x2e', '0xf2', '0x3f'], ['0x7b', '0xfc', '0xcc', '0xff', '0xac', '0x08', '0x29', '0x74', '0xac', '0xcd', '0xf5', '0x22', '0xc0', '0x19', '0x94', '0x84'], ['0xa2', '0xe7', '0x10', '0x0a', '0xa5', '0xad', '0x97', '0x60', '0x44', '0xde', '0x1e', '0x15', '0xd0', '0xa1', '0xe1', '0x57']]


In [94]:
array[0] = format_bytearray_to_hex(text1)
array[1] = format_bytearray_to_hex(correct_input)
array[2] = format_bytearray_to_hex(text3)
array[3] = format_bytearray_to_hex(text4)
print(array)

[['0x12', '0xe3', '0xfb', '0x36', '0x9a', '0x38', '0x7a', '0xbc', '0x89', '0xaa', '0xc3', '0xaa', '0x14', '0x1d', '0x96', '0xae'], ['0xa0', '0x27', '0xb9', '0x21', '0x0a', '0xc7', '0x7e', '0xca', '0xca', '0x16', '0x25', '0x2f', '0xc0', '0x2e', '0xf2', '0x3f'], ['0x7b', '0xfc', '0xcc', '0xff', '0xac', '0x08', '0x29', '0x74', '0xac', '0xcd', '0xf5', '0x22', '0xc0', '0x19', '0x94', '0x84'], ['0xa2', '0xe7', '0x10', '0x0a', '0xa5', '0xad', '0x97', '0x60', '0x44', '0xde', '0x1e', '0x15', '0xd0', '0xa1', '0xe1', '0x57']]


In [42]:
scope.glitch.offset = 36#glitch_setting[1]
scope.glitch.width = 22#glitch_setting[0]
scope.glitch.ext_offset = 40#glitch_setting[2]

#scope.arm()
target.simpleserial_write('b', text1)
#ret = scope.capture()

val = target.simpleserial_read_witherrors(timeout = 0,pay_len=32)
print(val)

gcnt = val['payload'][16:]
ipval = val['payload'][0:16]
print((gcnt != ct1) and (ipval == eight_input))

{'valid': True, 'payload': CWbytearray(b'a0 27 b9 21 0a c7 7e ca ca 16 25 2f c0 2e f2 3f 12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae'), 'full_response': CWbytearray(b'00 72 20 a0 27 b9 21 0a c7 7e ca ca 16 25 2f c0 2e f2 3f 12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae c7 00'), 'rv': bytearray(b'\x00')}
False


In [37]:
#12 e3 fb 36 9a 38 7a bc 89 aa c3 aa 14 1d 96 ae

# CWbytearray(b'7d 16 00 08 00 00 00 00 00 00 00 00 00 00 00 00')
# at 32.03125 16.015625 28
# 🐙Done glitching

#CWbytearray(b'12 e3 fb 36 9a 38 7a bc 89 aa c3 c9 14 1d 96 ae') #c9 , 4th from back is different
#at 32.03125 16.015625 37
#🐙Done glitching

# CWbytearray(b'7d 16 00 08 00 00 00 00 00 00 00 00 00 00 00 00')
# at 26.171875 21.875 28
# 🐙Done glitching

# CWbytearray(b'bd 49 58 af c9 ad 23 7d 05 f2 3c fb 34 ca 9c ef')
# at 26.171875 21.875 17
# 🐙Done glitching

# CWbytearray(b'00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00')
# at 17.96875 30.078125 33
# 🐙Done glitching

# CWbytearray(b'3a 2c 3e fe 69 16 73 0b 42 6a 71 54 93 57 7a c5')
# at 10.15625 37.890625 23
# 🐙

# CWbytearray(b'a7 ae 57 cd 0f 27 42 0b 7b 42 fe 83 85 2b 3a fd')
# glitchded at 32.03125 35.9375 28
# 🐙

In [1]:
scope.dis()
target.dis()

NameError: name 'scope' is not defined